In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import glob
import numpy as np

In [2]:
# Read in metadata (has all the vars we need for allom modeling!)
neon_meta_df = pd.read_csv("/data/shared/src/arojas/NEON/data/NEON_Field_Site_Metadata_20230309_Env_Vars.csv")
print(neon_meta_df.shape)
print(neon_meta_df.columns)

(81, 64)
Index(['field_domain_id', 'field_site_id', 'field_site_name',
       'field_site_type', 'field_site_subtype', 'field_colocated_site',
       'field_site_host', 'field_site_url', 'field_nonneon_research_allowed',
       'field_access_details', 'field_neon_field_operations_office',
       'field_latitude', 'field_longitude', 'field_geodetic_datum',
       'field_utm_northing', 'field_utm_easting', 'field_utm_zone',
       'field_site_county', 'field_site_state', 'field_site_country',
       'field_mean_elevation_m', 'field_minimum_elevation_m',
       'field_maximum_elevation_m', 'field_mean_annual_temperature_C',
       'field_mean_annual_precipitation_mm', 'field_dominant_wind_direction',
       'field_mean_canopy_height_m', 'field_dominant_nlcd_classes',
       'field_domint_plant_species', 'field_usgs_huc', 'field_watershed_name',
       'field_watershed_size_km2', 'field_lake_depth_mean_m',
       'field_lake_depth_max_m', 'field_tower_height_m',
       'field_usgs_geology_

In [3]:
# Lets read in the veg structure measurements with env vars
neon_veg = pd.read_csv("../../data/output/NEON_VST_TREES.csv")
print(neon_veg.shape)
# print(neon_veg.columns)

(167752, 45)


In [4]:
# Get hmax for each site from veg structure and add to df
for siteid, group in neon_veg.groupby("siteID"):
    print(siteid,end="\r")
    # get indices
    group_idx = neon_meta_df.loc[neon_meta_df['field_site_id']==siteid].index
    # Get 95 percentile height
    hmax = np.nanpercentile(group['height'].values,95)
    neon_meta_df.loc[group_idx, "site_hmax"] = hmax

In [5]:
# Lets add the scaling exponents for allometric relationships!
# Read allometry tables
h_d_df = pd.read_csv("../../data/output/site-allometry/NEON_site_allom_H_D.csv")
rc_d_df = pd.read_csv("../../data/output/site-allometry/NEON_site_allom_Rc_D.csv")
rc_h_df = pd.read_csv("../../data/output/site-allometry/NEON_site_allom_Rc_H.csv")
vc_h_df = pd.read_csv("../../data/output/site-allometry/NEON_site_allom_Vc_H.csv")

site_id_list = neon_meta_df['field_site_id'].unique()
for siteid in site_id_list:
    print(siteid,end="\r")
    # get indices
    group_idx = neon_meta_df.loc[neon_meta_df['field_site_id']==siteid].index
    if len(h_d_df.loc[h_d_df['siteID']==siteid])==0:
        continue
    # Get scaling exponents
    scaling_exp = h_d_df.loc[h_d_df['siteID']==siteid]['slope'].values[0]
    neon_meta_df.loc[group_idx, "H_D_pow"] = scaling_exp
    scaling_exp = rc_d_df.loc[rc_d_df['siteID']==siteid]['slope'].values[0]
    neon_meta_df.loc[group_idx, "Rc_D_pow"] = scaling_exp
    scaling_exp = rc_h_df.loc[rc_h_df['siteID']==siteid]['slope'].values[0]
    neon_meta_df.loc[group_idx, "Rc_H_pow"] = scaling_exp
    scaling_exp = vc_h_df.loc[vc_h_df['siteID']==siteid]['slope'].values[0]
    neon_meta_df.loc[group_idx, "Vc_H_pow"] = scaling_exp

In [7]:
neon_meta_df.columns

Index(['field_domain_id', 'field_site_id', 'field_site_name',
       'field_site_type', 'field_site_subtype', 'field_colocated_site',
       'field_site_host', 'field_site_url', 'field_nonneon_research_allowed',
       'field_access_details', 'field_neon_field_operations_office',
       'field_latitude', 'field_longitude', 'field_geodetic_datum',
       'field_utm_northing', 'field_utm_easting', 'field_utm_zone',
       'field_site_county', 'field_site_state', 'field_site_country',
       'field_mean_elevation_m', 'field_minimum_elevation_m',
       'field_maximum_elevation_m', 'field_mean_annual_temperature_C',
       'field_mean_annual_precipitation_mm', 'field_dominant_wind_direction',
       'field_mean_canopy_height_m', 'field_dominant_nlcd_classes',
       'field_domint_plant_species', 'field_usgs_huc', 'field_watershed_name',
       'field_watershed_size_km2', 'field_lake_depth_mean_m',
       'field_lake_depth_max_m', 'field_tower_height_m',
       'field_usgs_geology_unit', 'f

In [8]:
# Calculate HSE
# (1+2/beta)-alpha
neon_meta_df['HSE'] = (1+2/neon_meta_df['H_D_pow'].values) - neon_meta_df['Vc_H_pow'].values
# Save HSE for each site!
# neon_meta_df[['field_domain_id', 'field_site_id',
#               'site_hmax',
#               'H_D_pow', 'Rc_D_pow',
#               'Rc_H_pow','Vc_H_pow']].to_csv("../../data/output/neon-sites-allom-HSE.csv",index=False)



In [9]:
##############
## Allometric modeling
##############

In [10]:
# Now that we have all structural, plot level, and environmental variables, lets model!

In [11]:
from sklearn.ensemble import RandomForestRegressor
from matplotlib.colors import to_rgba
from sklearn.metrics import mean_squared_error
import seaborn as sns
sns.set_theme(style="whitegrid", font_scale=1.35, rc={'grid.color': '#ededed'})

outdir = "allyears"

# Getting input data
# lets extract the columns
data_cols = ['wc_srad_avg_yr',
       'wc_vapr_avg_yr', 'wc_wind_avg_yr', 'wc_bio_1', 'wc_bio_12',
       'wc_bio_15', 'pet_he_yr', 'ai_yr', 'E', 'CWD', 'sg_cec_avg',
       'sg_clay_avg', 'sg_phh2o_avg', 'sg_sand_avg', 'sg_silt_avg',
       'site_hmax',"field_mean_elevation_m"]
# independent and dependent vars
allom_substrs_list = ["H_D","Rc_D","Rc_H","Vc_H"]
for allom_substr in allom_substrs_list:
    # allom_substr = "H_D"
    xy_data_arr = np.hstack([neon_meta_df[data_cols].values,
                            neon_meta_df[f"{allom_substr}_pow"].values.reshape(-1,1)])
    nodata_bool = ~np.isnan(xy_data_arr).any(axis=1)
    xy_data_arr = xy_data_arr[nodata_bool]
    X_data_arr = xy_data_arr[:,:-1]
    y_data_arr = xy_data_arr[:,-1]
    siteids_filt = neon_meta_df["field_site_id"].values[nodata_bool]
    
    # Get feature importance
    forest_classifier = RandomForestRegressor(random_state=8)
    forest_classifier.fit(X_data_arr,y_data_arr)
    
    feature_imp_df = pd.DataFrame({"score":forest_classifier.feature_importances_}, index=data_cols)
    feature_imp_df = feature_imp_df.sort_values(by="score", ascending=False)
    sns.set_context("notebook", font_scale=1.2)
    fig,ax = plt.subplots(1,1,figsize=(10,6))
    ax.set_title(f"{allom_substr} Allometry Random Forest Relative Importance")
    sns.barplot(x=feature_imp_df['score'], y=feature_imp_df.index, ax=ax)
    
    ## Add inset plot with errors and eval metrics
    left, bottom, width, height = [0.6, 0.25, 0.35, 0.4] # 0,0 is bottom left)
    ax2 = fig.add_axes([left, bottom, width, height])
    # Plot residuals vs fitted
    y_pred = forest_classifier.predict(X_data_arr)
    residuals = y_data_arr - y_pred
    ax2.scatter(y_pred, residuals,
                 color=to_rgba("#1f77b4", .3),
                 ec="#1f77b4", lw=0.5)
    ax2.axhline(0, color="#7f7f7f", ls="--")
    ax2.set_xlabel("Predicted")
    ax2.set_ylabel("Residual")
    # Fix yrange
    yrange_max = np.max(np.absolute(ax2.get_ylim()))
    ax2.set_ylim(yrange_max*-1,yrange_max)
    # Add metrics text
    r2 = forest_classifier.score(X_data_arr, y_data_arr)
    rmse = mean_squared_error(y_true=y_data_arr.reshape(-1,1),
                              y_pred=y_pred.reshape(-1,1), squared=False)
    txt = f"R2={np.round(r2,2)}\nRMSE={np.round(rmse,2)}"
    ax2.text(0.98, 0.0125, txt, ha='right', va='bottom',
             transform=ax2.transAxes)
    
    fig.tight_layout()
    fig.savefig(f"../../figs/veg-struct/allom/{outdir}/NEON_site_allom_{allom_substr}_FeatImportance.png", dpi=300)
    plt.close()
    

/tmp/ipykernel_3231711/2863786260.py:62: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()
/tmp/ipykernel_3231711/2863786260.py:62: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()
/tmp/ipykernel_3231711/2863786260.py:62: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()
/tmp/ipykernel_3231711/2863786260.py:62: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()


In [13]:
from sklearn.ensemble import RandomForestRegressor
from matplotlib.colors import to_rgba
from sklearn.metrics import mean_squared_error
import seaborn as sns
sns.set_theme(style="whitegrid", font_scale=1.35, rc={'grid.color': '#ededed'})

outdir="allyears"

# Getting input data
# lets extract the columns
data_cols = ['wc_srad_avg_yr',
       'wc_vapr_avg_yr', 'wc_wind_avg_yr', 'wc_bio_1', 'wc_bio_12',
       'wc_bio_15', 'pet_he_yr', 'ai_yr', 'E', 'CWD', 'sg_cec_avg',
       'sg_clay_avg', 'sg_phh2o_avg', 'sg_sand_avg', 'sg_silt_avg',
       'site_hmax',"field_mean_elevation_m"]
# independent and dependent vars
allom_substrs_list = ["HSE"]
for allom_substr in allom_substrs_list:
    # allom_substr = "H_D"
    xy_data_arr = np.hstack([neon_meta_df[data_cols].values,
                            neon_meta_df[allom_substr].values.reshape(-1,1)])
    nodata_bool = ~np.isnan(xy_data_arr).any(axis=1)
    xy_data_arr = xy_data_arr[nodata_bool]
    X_data_arr = xy_data_arr[:,:-1]
    y_data_arr = xy_data_arr[:,-1]
    siteids_filt = neon_meta_df["field_site_id"].values[nodata_bool]
    
    # Get feature importance
    forest_classifier = RandomForestRegressor(random_state=8)
    forest_classifier.fit(X_data_arr,y_data_arr)
    
    feature_imp_df = pd.DataFrame({"score":forest_classifier.feature_importances_}, index=data_cols)
    feature_imp_df = feature_imp_df.sort_values(by="score", ascending=False)
    sns.set_context("notebook", font_scale=1.2)
    fig,ax = plt.subplots(1,1,figsize=(10,6))
    ax.set_title(f"NEON Sites {allom_substr} Random Forest Relative Importance")
    sns.barplot(x=feature_imp_df['score'], y=feature_imp_df.index, ax=ax)
    
    ## Add inset plot with errors and eval metrics
    left, bottom, width, height = [0.6, 0.25, 0.35, 0.4] # 0,0 is bottom left)
    ax2 = fig.add_axes([left, bottom, width, height])
    # Plot residuals vs fitted
    y_pred = forest_classifier.predict(X_data_arr)
    residuals = y_data_arr - y_pred
    ax2.scatter(y_pred, residuals,
                 color=to_rgba("#1f77b4", .3),
                 ec="#1f77b4", lw=0.5)
    ax2.axhline(0, color="#7f7f7f", ls="--")
    ax2.set_xlabel("Predicted")
    ax2.set_ylabel("Residual")
    # Fix yrange
    yrange_max = np.max(np.absolute(ax2.get_ylim()))
    ax2.set_ylim(yrange_max*-1,yrange_max)
    # Add metrics text
    r2 = forest_classifier.score(X_data_arr, y_data_arr)
    rmse = mean_squared_error(y_true=y_data_arr.reshape(-1,1),
                              y_pred=y_pred.reshape(-1,1), squared=False)
    txt = f"R2={np.round(r2,2)}\nRMSE={np.round(rmse,2)}"
    ax2.text(0.98, 0.0125, txt, ha='right', va='bottom',
             transform=ax2.transAxes)
    
    fig.tight_layout()
    fig.savefig(f"../../figs/veg-struct/allom/{outdir}/NEON_site_HSE_FeatImportance.png", dpi=300)
    plt.close()
    

/tmp/ipykernel_3231711/1040146890.py:62: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()
